<a href="https://colab.research.google.com/github/okunkov/park_view/blob/master/Reketa_PROD_v2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Заполни дату
# @markdown Дата начала выписки

date = '2023-11-06'  # @param {type: "date"}

from datetime import datetime, timezone


def string_to_isotime(date_string):
    datetime_object = datetime.fromisoformat(date_string)
    return datetime_object.strftime("%Y-%m-%dT%H:%M:%S.000Z")


# datetime_object = datetime.strptime(date, '%Y-%m-%d')
# iso_date = datetime_object.strftime("%Y-%m-%dT%H:%M:%S.000Z")

DATE_FROM = string_to_isotime(date)
DATE_NOW = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
print(f"{DATE_FROM = }\n{DATE_NOW = }")


DATE_FROM = '2023-11-06T00:00:00.000Z'
DATE_NOW = '2023-11-10T18:24:37.000Z'


In [ ]:
#@title Загрузи .xlsx файл
#@markdown

from google.colab import files
uploaded = files.upload()





Saving Петровакс_выгрузка_их_МОМ_как_есть.xls to Петровакс_выгрузка_их_МОМ_как_есть.xls


In [ ]:
# @title Запускай
# @markdown


import requests
import pandas as pd
import re
import io
from time import sleep
# from tables import file


#  DEPRICATED !!!!!!!!!!!!!!!!!!!!!!!!!!
def parse_ticket_number(string_data):
    '''
        Parse ticket number from text
        *** в API к номеру билеты прибавляется код компании - пример 'number': '5556177886145'
    '''
    result = re.search(r"\d{3}\s\d{10} | \d{14}", string_data)
    if result:
        result = result.group().replace(" ", "")
        return result
    else:
        print('Номер билета не найден')
        return None

## TEST
# excel_string = 'Авиабилет МАЦИЯК МАКСИМ ВИКТОРОВИЧ  425 6110231540  Москва - Самара - Москва    28.10.23'
# print(parse_ticket_number(excel_string))


def find_order_client_id(ticket_number, json):
    '''
    Get order id by ticket number
    '''
    result = [None, None]

    if not (ticket_number and json):
        print('Не указан номер билета, ошибка функции')
        return result

    for element in json:
        if element['type'] == 'avia' \
            and element['avia']['tickets'] \
            and element['avia']['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['avia']['tickets'][0]['passengerId']]
            return result
        if element['type'] == 'rail' \
            and element['rail']['forward'][0]['tickets'] \
            and element['rail']['forward'][0]['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['rail']['forward'][0]['tickets'][0]['passengerId']]
            return result

    return result

## TEST
# print(find_order_client_id('5556179106551'))


def find_tabnum_cfo_code(client_id, headers):
    '''
    Найти ЦФО, Код ЦФО и табельный номер
    '''
    if client_id:
        client_id = str(client_id)
        profiles_endpoint = 'https://agency.raketa.travel/v2/profiles/' + client_id
        json_data_profiles = requests.get(profiles_endpoint, headers=headers).json()

        if json_data_profiles['extrafields']:
            return [json_data_profiles['code'], json_data_profiles['costCenter'], json_data_profiles['extrafields'][0]['value']]
        else:
            return [json_data_profiles['code'], json_data_profiles['costCenter'], '']
    else:
        print('ЦФО, код ЦФО и таб.номер - не найден')
        return [None, None, None]
## TEST
# order_id, client_id = '5356397', '1548785'
# print(find_tabnum_cfo_code(client_id))


def find_brand(order_id, headers):
    '''
    Поиска Бренда
    '''
    if order_id:
        order_id = str(order_id)
        brand_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}'
        json_data_brand = requests.get(brand_endpoint, headers=headers).json()

        if json_data_brand['extrafields']:
            ''' С номером элеманта массива [1] могут быть *фокусы* '''
            return str(json_data_brand['extrafields'][1]['value'])
    else:
        print('Бренд - не найден')
        return None

## TEST
# print(find_brand(order_id))


def find_businesstrip(order_id, headers):
    '''
        Поиск Цель командировки
    '''
    if order_id:
        order_id = str(order_id)
        profiles_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}/businessTrip'
        json_data_businesstrip = requests.get(profiles_endpoint, headers=headers).json()
        if json_data_businesstrip['purpose']:
            return str(json_data_businesstrip['purpose'])
    else:
        print('Цель командировки - не найдена')
        return None

# TEST
# print(find_businesstrip(5356397))


def main():
    '''
        MAIN FUNCTION
    '''
    filename = list(uploaded.keys())
    print('Excel filename is:', filename)

    df = pd.read_excel(io.BytesIO(uploaded[filename[0]]), header=[2])
    # print(df.columns)
    # print(df.head())

    # VARS
    date_from = DATE_FROM
    token = 'c52a73d26c1a487901da43f1c30bdf37ff98adfae2bdf3574efa10cd4249e55d'
    headers = {'Authorization': f'Bearer {token}', 'accept': 'application/json'}

    services_endpoint = 'https://agency.raketa.travel/v2/services?fromLastUpdate=' + date_from
    json_data_services = requests.get(services_endpoint, headers=headers).json()
    json_last_date = json_data_services[len(json_data_services)-1]['lastUpdate']

    while json_last_date[:10] != DATE_NOW[:10]:
        date_from = string_to_isotime(json_last_date)
        url = "https://agency.raketa.travel/v2/services?fromLastUpdate=" + date_from
        new_request = requests.get(url, headers=headers).json()
        if not new_request: break

        print(f'New request date: {date_from} {len(new_request) = }')

        json_data_services.extend(new_request)
        json_last_date = new_request[len(new_request)-1]['lastUpdate']
        print(f'{json_data_services[len(json_data_services)-1]["lastUpdate"]}')



    for index, row in df.iterrows():
        data_string = row[1]
        print(f"{'='*80} \n {index + 1}: {data_string}")

        ticket_number = parse_ticket_number(data_string)
        if ticket_number:
            order_id, client_id = find_order_client_id(ticket_number, json_data_services)
            tabnum, cfo, cfo_code = find_tabnum_cfo_code(client_id, headers)
            businesstrip = find_businesstrip(order_id, headers)
            brand = find_brand(order_id, headers)

            if len(ticket_number) == 13:
                ticket_number = ticket_number[3::]

            print(f"{'№ СОБ' : <10} {order_id}\n"
                f"{'№ билета' : <10} {ticket_number}\n"
                f"{'Таб.номер' : <10} {tabnum}\n"
                f"{'ЦФО' : <10} {cfo}\n"
                f"{'Код ЦФО' : <10} {cfo_code}\n"
                f"{'Цель ком.' : <10} {businesstrip}\n"
                f"{'Бренд' : <10} {brand}\n")

            # Распчатываем TAX
            #    f"{'Бренд' : <10} {brand}\n")


            # Petrovax Add to DataFrame
            df.at[index, '№ заказа в СОБ'] = order_id
            df.at[index, 'Номер билета'] = ticket_number
            df.at[index, 'Табельный номер'] = tabnum
            df.at[index, 'ЦФО'] = cfo
            df.at[index, 'Код ЦФО'] = cfo_code
            df.at[index, 'Цель командировки'] = businesstrip
            df.at[index, 'Бренд'] = brand

            # sleep(1)

    # Запись измененного DataFrame в файл Excel
    # print(filename)
    df.to_excel('Petrovax_report.xlsx', index=False)

                # VERVAG Add to DataFrame
            # df.at[index, 'Кост-центр путешественника'] = tabnum
            # df.at[index, 'Цель поездки'] = businesstrip
            # df.at[index, 'Препарат'] = brand

            # sleep(1)

    # Запись измененного DataFrame в файл Excel Vervag
    # print(filename)
    # df.to_excel('Vervag_report.xlsx', index=False)


if __name__ == '__main__':
    main()

Excel filename is: ['Петровакс_выгрузка_их_МОМ_как_есть.xls']
New request date: 2023-11-08T21:50:28.000Z len(new_request) = 54
2023-11-10T21:15:06+03:00
 1: nan


TypeError: ignored

### ТЕСТИРОВАНИЕ

In [ ]:

# STAGE
# url = 'https://agency.master.raketa.online/v2'

# PROD
# url = 'https://agency.raketa.travel/v2'


# ## Test
import re

import requests
from pprint import pprint

# token = 'c52a73d26c1a487901da43f1c30bdf37ff98adfae2bdf3574efa10cd4249e55d'
# headers = {'Authorization': f'Bearer {token}', 'accept': 'application/json'}
# date_from = '2023-09-20T00:00:00.000Z'
# services_endpoint = 'https://agency.raketa.travel/v2/services?fromLastUpdate=' + date_from
# json_data_services = requests.get(services_endpoint, headers=headers).json()

# pprint(json_data_services)

# for element in json_data_services:
#     if element['type'] == 'rail' and element['rail']['forward']:
#         print('\n*************************')

#         print(f"{element['rail']['forward'][0]['tickets'][0]['number'] = }")
#         print(f"{element['rail']['forward'][0]['tickets'][0]['passengerId'] = }")
#         print(f"{element['orderId'] = }")

#         # pprint(element['rail']['backward'][0]['tickets'][0]['number'])

#         # print(f"{element['orderId']} - orderId (номер заказа в СОБ)")
#         # print(f"{element['avia']['tickets'][0]['number']} - number (номер билета)")
#         # print(f"{element['avia']['tickets'][0]['passengerName']}, {element['avia']['tickets'][0]['passengerId']} - passengerName | ID (ФИО)")



# def parse_ticket_number(string_data):
#     '''
#         Parse ticket number from text
#         *** в API к номеру билеты прибавляется код компании - пример 'number': '5556177886145'
#     '''
#     result = re.search(r"\d{3}\s\d{10} | \d{14}", string_data)
#     if result:
#         result = result.group().replace(" ", "")
#         return result
#     else:
#         print('Номер билета не найден')
#         return None

# ## TEST
# s0 = "Авиабилет МАЦИЯК МАКСИМ ВИКТОРОВИЧ  425 6110231540  Москва - Самара - Москва    28.10.23"
# s1 = "Жд билет Макеева Лидия Александровна        75884767714044  ПЯТИГОРСК-КРАСНОДАР 1   22.08.23"
# s2 = "Жд билет Макеева Лидия Александровна        75934767719751  КРАСНОДАР 1-ВОЛГОГРАД 1 24.08.23"
# s3 = "ЖПроживание в отеле Краснодар    Hilton Garden Inn Krasnodar (Хилтон Гарден Инн Краснодар), Краснодар    Макеева Лидия Александровна Номер стандарт с 1 двуспальной кроватью 22.08.2023 14:00    24.08.2023 12:00"
# print(parse_ticket_number(s3))


Номер билета не найден
None
